# Chapter 21: CI with Docker

While Chapters 11 and 19 established how container images are stored and how CI workflows trigger, this chapter examines the critical intersection where source code transforms into containerized artifacts. Docker in CI presents unique challenges: ephemeral build environments, layer caching optimization, security constraints, and the architectural decisions between Docker-in-Docker versus alternative build strategies.

Modern CI pipelines must produce container images efficiently, securely, and reproducibly across diverse infrastructures—from cloud-managed runners to self-hosted Kubernetes agents. Understanding how to optimize Docker builds for automated environments reduces pipeline duration, minimizes storage costs, and ensures consistent artifacts from development through production.

## 21.1 Building Images in CI

Building container images within automated pipelines requires understanding the execution context, build environment constraints, and optimization techniques specific to ephemeral CI runners.

### The Build Context Challenge

CI runners operate in clean, ephemeral environments where the build context (files sent to the Docker daemon) significantly impacts performance:

```dockerfile
# .dockerignore - Critical for CI performance
# Exclude version control
.git
.gitignore
.gitattributes

# Exclude CI/CD configurations not needed in image
.github/
.gitlab-ci.yml
.azure-pipelines.yml

# Exclude local development files
*.md
docker-compose*.yml
.env
.env.local
.env.*.local

# Exclude IDE configurations
.idea/
.vscode/
*.swp
*.swo

# Exclude test artifacts (if tests run before build)
coverage/
.nyc_output/
*.log
logs/

# Exclude node_modules (will be installed in container)
node_modules/
vendor/
target/  # Rust/Java build artifacts
__pycache__/
*.pyc
```

**Impact Analysis:**
A typical Node.js project might have:
- Source code: 5 MB
- `node_modules`: 500 MB
- `.git`: 50 MB
- Without `.dockerignore`: 555 MB context transfer per build
- With proper `.dockerignore`: 5 MB context transfer (99% reduction)

### Basic CI Build Patterns

**GitHub Actions with Docker Build:**
```yaml
name: Build Container Image

on:
  push:
    branches: [main, develop]
  pull_request:
    branches: [main]

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout Repository
        uses: actions/checkout@v4
        with:
          fetch-depth: 1  # Shallow clone for faster checkout

      - name: Set up Docker Buildx
        uses: docker/setup-buildx-action@v3

      - name: Build Image
        run: |
          docker build \
            --file Dockerfile \
            --tag myapp:${{ github.sha }} \
            --tag myapp:latest \
            .

      - name: Test Image Structure
        run: |
          docker run --rm myapp:${{ github.sha }} echo "Container starts successfully"
          docker image inspect myapp:${{ github.sha }} --format='{{.Size}}'
```

**GitLab CI with Kaniko (Rootless Alternative):**
```yaml
build:
  stage: build
  image:
    name: gcr.io/kaniko-project/executor:debug
    entrypoint: [""]
  script:
    - /kaniko/executor
      --context "${CI_PROJECT_DIR}"
      --dockerfile "${CI_PROJECT_DIR}/Dockerfile"
      --destination "${CI_REGISTRY_IMAGE}:${CI_COMMIT_SHA}"
      --destination "${CI_REGISTRY_IMAGE}:latest"
      --cache=true
      --cache-ttl=24h
```

### Runner Requirements and Constraints

**Docker Socket Access (DooD - Docker-outside-of-Docker):**
Mounting the host Docker socket provides native performance but requires privileged access:

```yaml
# GitLab CI with Docker socket binding
build:
  stage: build
  image: docker:24
  services:
    - docker:24-dind  # Docker-in-Docker service
  variables:
    DOCKER_TLS_CERTDIR: "/certs"
    DOCKER_DRIVER: overlay2
  before_script:
    - docker info
  script:
    - docker build -t $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA .
```

**Security Implications:**
- Docker socket access effectively grants root access to the host
- Any container can escape to the host via the mounted socket
- Mitigation: Use dedicated, isolated build nodes or rootless alternatives

## 21.2 Multi-Stage Builds in CI

Multi-stage builds are essential for CI/CD pipelines, enabling the separation of build-time dependencies from runtime artifacts, resulting in smaller, more secure production images.

### The CI Build Pattern

```dockerfile
# Dockerfile optimized for CI/CD
# Stage 1: Dependencies and Build
FROM node:20-alpine AS builder
WORKDIR /app

# Copy dependency manifests first (layer caching)
COPY package*.json ./
RUN npm ci --only=production=false --ignore-scripts

# Copy source and build
COPY . .
RUN npm run build
RUN npm prune --production

# Stage 2: Production Runtime
FROM node:20-alpine AS production
WORKDIR /app
ENV NODE_ENV=production

# Security: Run as non-root
RUN addgroup -g 1001 -S nodejs && \
    adduser -S nodejs -u 1001

# Copy only necessary artifacts from builder
COPY --from=builder --chown=nodejs:nodejs /app/dist ./dist
COPY --from=builder --chown=nodejs:nodejs /app/node_modules ./node_modules
COPY --from=builder --chown=nodejs:nodejs /app/package.json ./package.json

USER nodejs

EXPOSE 3000
HEALTHCHECK --interval=30s --timeout=3s --start-period=5s --retries=3 \
    CMD node -e "require('http').get('http://localhost:3000/health', (r) => r.statusCode === 200 ? process.exit(0) : process.exit(1))"

CMD ["node", "dist/server.js"]
```

**CI Advantages:**
1. **Faster Push/Pull**: Production image is ~50MB vs ~500MB with dev dependencies
2. **Reduced Attack Surface**: No compilers, build tools, or source maps in production
3. **Layer Reuse**: Base image layers cache independently of application code changes

### Language-Specific CI Optimizations

**Java/Maven Multi-Stage:**
```dockerfile
# Stage 1: Build with all Maven dependencies
FROM maven:3.9-eclipse-temurin-21-alpine AS build
WORKDIR /workspace
COPY pom.xml .
# Download dependencies (cached layer)
RUN mvn dependency:go-offline -B
COPY src ./src
RUN mvn package -DskipTests -B

# Stage 2: Runtime with JRE only
FROM eclipse-temurin:21-jre-alpine
COPY --from=build /workspace/target/*.jar app.jar
ENTRYPOINT ["java", "-jar", "/app.jar"]
```

**Python with Poetry:**
```dockerfile
FROM python:3.11-slim AS builder
WORKDIR /app
RUN pip install poetry
COPY pyproject.toml poetry.lock ./
RUN poetry config virtualenvs.create false && \
    poetry install --no-dev --no-interaction --no-ansi

FROM python:3.11-slim AS production
WORKDIR /app
COPY --from=builder /usr/local/lib/python3.11/site-packages /usr/local/lib/python3.11/site-packages
COPY --from=builder /usr/local/bin /usr/local/bin
COPY ./src ./src
CMD ["python", "-m", "src.main"]
```

### Distroless and Minimal Images

For maximum security in CI-produced images:

```dockerfile
# Go application with distroless
FROM golang:1.21-alpine AS build
WORKDIR /src
COPY go.mod go.sum ./
RUN go mod download
COPY . .
RUN CGO_ENABLED=0 GOOS=linux go build -a -installsuffix cgo -o app .

FROM gcr.io/distroless/static:nonroot
COPY --from=build /src/app /app
USER 65532:65532
ENTRYPOINT ["/app"]
```

**Benefits:**
- No shell, package manager, or utilities (no `sh`, `curl`, `apt`)
- Reduces CVE surface area by ~80%
- Forces explicit dependency declaration (cannot "debug" in container)

## 21.3 Caching Docker Layers

Efficient layer caching distinguishes fast CI pipelines from slow ones. Without caching, every build reinstalls dependencies and recompiles code, wasting time and compute resources.

### Registry-Based Caching

Leverage the container registry as a cache backend:

```yaml
# GitHub Actions with registry cache
- name: Build and push
  uses: docker/build-push-action@v5
  with:
    context: .
    push: true
    tags: |
      ghcr.io/${{ github.repository }}:${{ github.sha }}
      ghcr.io/${{ github.repository }}:latest
    cache-from: type=registry,ref=ghcr.io/${{ github.repository }}:buildcache
    cache-to: type=registry,ref=ghcr.io/${{ github.repository }}:buildcache,mode=max
```

**Mode Options:**
- `mode=min`: Cache only final layers (smaller, faster)
- `mode=max`: Cache all intermediate layers (better for multi-stage, larger storage)

### GitHub Actions Cache Backend

For private repositories without registry cache support:

```yaml
- name: Build with GHA cache
  uses: docker/build-push-action@v5
  with:
    context: .
    push: false
    load: true  # Load to Docker daemon for testing
    tags: myapp:test
    cache-from: type=gha
    cache-to: type=gha,mode=max
```

**Cache Invalidation Strategy:**
Docker layer caching is based on instruction content and previous layer hash. To force cache refresh:

```dockerfile
# Add cache-busting argument
ARG CACHEBUST=1
RUN apt-get update && apt-get install -y ...
```

In CI:
```yaml
--build-arg CACHEBUST=$(date +%s)
```

### Layer Ordering Optimization

Structure Dockerfile for maximum cache hits during iterative development:

```dockerfile
# GOOD: Dependencies before source code
COPY package*.json ./
RUN npm ci  # Cached unless package.json changes
COPY . .    # This layer changes frequently
RUN npm run build

# BAD: Source code before dependencies
COPY . .    # Changes on every commit, invalidates subsequent layers
RUN npm ci  # Never cached effectively
```

### Local Cache with Bind Mounts (Advanced)

For self-hosted runners with persistent storage:

```yaml
# GitLab CI with host cache directory
build:
  script:
    - docker build 
      --cache-from type=local,src=/cache/docker 
      --cache-to type=local,dest=/cache/docker,mode=max 
      -t myapp:$CI_COMMIT_SHA .
  cache:
    key: docker-layers
    paths:
      - /cache/docker
```

## 21.4 Docker-in-Docker Considerations

Docker-in-Docker (DinD) refers to running Docker containers inside Docker containers, a common pattern in CI but with significant security and performance implications.

### The DinD Architecture

```yaml
# GitLab CI Docker-in-Docker configuration
build:
  image: docker:24
  services:
    - docker:24-dind
  variables:
    DOCKER_TLS_CERTDIR: "/certs"  # Enable TLS between client and server
    DOCKER_DRIVER: overlay2
  script:
    - docker build -t myapp:latest .
```

**How It Works:**
1. CI runner starts `docker:24` container (client)
2. Service container `docker:24-din` starts (daemon)
3. Client communicates with daemon via TCP/TLS on `docker:2376`
4. Images built inside the service container's filesystem

### Security Concerns

**Privileged Mode:**
DinD requires privileged containers (`--privileged`), which:
- Disable all security isolation mechanisms
- Grant full host device access
- Allow kernel manipulation
- Enable container escape to host

**Mitigation Strategies:**

1. **Use Kaniko (Google):**
   Rootless building without Docker daemon:
   ```yaml
   build:
     image:
       name: gcr.io/kaniko-project/executor:debug
       entrypoint: [""]
     script:
       - /kaniko/executor
         --context $CI_PROJECT_DIR
         --dockerfile $CI_PROJECT_DIR/Dockerfile
         --destination $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA
         --cache-dir /cache/kaniko
   ```

2. **Use Buildah (Red Hat):**
   OCI-compliant building without root:
   ```yaml
   build:
     image: quay.io/buildah/stable:latest
     script:
       - buildah bud -t myapp:latest .
       - buildah push myapp:latest $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA
   ```

3. **Docker Socket Binding (DooD):**
   Mount host socket with read-only restrictions (still requires careful isolation):
   ```yaml
   build:
     image: docker:24
     volumes:
       - /var/run/docker.sock:/var/run/docker.sock:ro
     script:
       - docker build -t myapp:latest .
   ```

### Performance Considerations

**Storage Driver Mismatch:**
If the DinD container uses `overlay2` but the host uses `vfs`, performance degrades significantly. Always match storage drivers or use volume mounts for `/var/lib/docker`.

**Layer Cache Isolation:**
DinD starts with empty cache on each job unless using registry cache or external volume mounts. This results in slower builds but cleaner environments.

## 21.5 BuildKit Integration

BuildKit is Docker's modern builder backend, offering enhanced performance, advanced caching, and security features essential for production CI pipelines.

### Enabling BuildKit

```bash
# Environment variable (CI configuration)
export DOCKER_BUILDKIT=1

# Or in daemon.json
{
  "features": {"buildkit": true}
}
```

### Advanced BuildKit Features in CI

**Secret Mounts (Avoid leaking credentials in layers):**
```dockerfile
# Dockerfile
FROM node:20-alpine
RUN --mount=type=secret,id=npmrc,target=/root/.npmrc \
    npm ci
```

```yaml
# CI Pipeline
- run: |
    echo "//registry.npmjs.org/:_authToken=${NPM_TOKEN}" > npmrc
    docker build \
      --secret id=npmrc,src=npmrc \
      -t myapp:latest .
```

**SSH Mounts (Private Git dependencies):**
```dockerfile
FROM golang:1.21
RUN --mount=type=ssh \
    go mod download
```

```bash
docker build --ssh default=${SSH_AUTH_SOCK} -t myapp:latest .
```

**Cache Mounts (Persistent package caches):**
```dockerfile
RUN --mount=type=cache,target=/var/cache/apt,sharing=locked \
    apt-get update && apt-get install -y python3
```

**Parallel Multi-Platform Builds:**
```yaml
- name: Build multi-platform
  uses: docker/build-push-action@v5
  with:
    platforms: linux/amd64,linux/arm64
    push: true
    tags: myapp:latest
```

### Dockerfile Syntax Directives

Enable experimental features:

```dockerfile
# syntax=docker/dockerfile:1.6
FROM node:20-alpine
# Now supports heredocs, --link, and other features
COPY --link package.json ./
```

## 21.6 Image Tagging in CI

Consistent, meaningful tagging in CI enables traceability, rollback capabilities, and clear promotion pathways through environments.

### Immutable Tagging Strategy

Never overwrite tags in CI; each build produces unique identifiers:

```yaml
# Generate comprehensive tags
- name: Generate tags
  id: meta
  uses: docker/metadata-action@v5
  with:
    images: ghcr.io/${{ github.repository }}
    tags: |
      type=sha,prefix=,suffix=,format=short
      type=ref,event=branch
      type=semver,pattern={{version}}
      type=raw,value=latest,enable={{is_default_branch}}

# Result:
# - sha: abc1234 (immutable, traceable)
# - branch: main (floating, latest on branch)
# - semver: v1.2.3 (if tag pushed)
# - latest (only on default branch)
```

### Tagging Conventions for CI/CD Flow

```bash
# Build phase
docker build -t myapp:git-${GIT_COMMIT_SHA:0:7} \
             -t myapp:build-${BUILD_NUMBER} \
             -t myapp:${BRANCH_NAME//\//-} .  # Replace / with - for safety

# Push all tags
docker push myapp --all-tags

# Promotion (retagging without rebuild)
docker pull myapp:git-abc1234
docker tag myapp:git-abc1234 myapp:staging
docker push myapp:staging
```

### Avoiding the Latest Tag in Production CI

While `latest` is convenient for development, CI pipelines should avoid it for production deployments:

```yaml
# Anti-pattern (avoid)
docker build -t myapp:latest .
docker push myapp:latest
# helm upgrade --set image.tag=latest (DANGEROUS - non-deterministic)

# Best Practice
docker build -t myapp:${GIT_SHA} .
docker push myapp:${GIT_SHA}
# helm upgrade --set image.tag=${GIT_SHA} (SAFE - immutable)
```

**Exception:** Use `latest` only for:
- Local development environments
- Base image updates (when intentionally tracking upstream)
- Documentation/examples where specific versions don't matter

## 21.7 Registry Authentication

CI pipelines must authenticate to registries to push images and optionally pull private base images.

### Short-Lived Token Authentication

**AWS ECR:**
```bash
# Get token valid for 12 hours
aws ecr get-login-password --region us-east-1 | \
  docker login --username AWS --password-stdin 123456789012.dkr.ecr.us-east-1.amazonaws.com
```

**Azure ACR:**
```bash
# Service Principal authentication
docker login myregistry.azurecr.io \
  --username $SP_CLIENT_ID \
  --password $SP_CLIENT_SECRET
```

**Google Artifact Registry:**
```bash
# Access token (60 min validity)
gcloud auth print-access-token | \
  docker login -u oauth2accesstoken --password-stdin https://us-central1-docker.pkg.dev
```

### Credential Helper Configuration

Configure Docker to use cloud provider helpers:

```json
// ~/.docker/config.json
{
  "credHelpers": {
    "123456789012.dkr.ecr.us-east-1.amazonaws.com": "ecr-login",
    "gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud",
    "myregistry.azurecr.io": "azurecr"
  }
}
```

**GitHub Actions Example:**
```yaml
- name: Configure AWS Credentials
  uses: aws-actions/configure-aws-credentials@v4
  with:
    role-to-assume: arn:aws:iam::123456789012:role/ECRPushRole
    aws-region: us-east-1

- name: Login to Amazon ECR
  id: login-ecr
  uses: aws-actions/amazon-ecr-login@v2

- name: Build, tag, and push
  env:
    ECR_REGISTRY: ${{ steps.login-ecr.outputs.registry }}
  run: |
    docker build -t $ECR_REGISTRY/myapp:$GITHUB_SHA .
    docker push $ECR_REGISTRY/myapp:$GITHUB_SHA
```

### Kubernetes Integration

Generate imagePullSecrets from CI:

```bash
# Create secret from Docker config
kubectl create secret docker-registry regcred \
  --docker-server=$REGISTRY \
  --docker-username=$USERNAME \
  --docker-password=$PASSWORD \
  --dry-run=client -o yaml | kubectl apply -f -
```

## 21.8 Build Failures and Debugging

When Docker builds fail in CI, debugging requires specific techniques due to the ephemeral nature of runners.

### Common CI Build Failures

**Context Timeout:**
```bash
# Error: "Build context is too large"
# Solution: Optimize .dockerignore
du -sh . | grep -E "^\d{3,}M" && echo "Context too large" || echo "Context OK"
```

**Out of Memory:**
```bash
# Error: "executor failed running..." or killed process
# Solution: Increase runner memory or reduce parallelism
docker build --memory=2g --memory-swap=2g .
```

**Layer Cache Misses:**
```bash
# Build takes too long, not using cache
# Diagnosis: Check if previous layers exist
docker build --progress=plain -t test . 2>&1 | grep -E "(importing cache|CACHED|RUN)"
```

### Debugging Techniques

**Verbose Logging:**
```bash
docker build --progress=plain --no-cache -t debug .
```

**Inspecting Intermediate Layers:**
```bash
# Build until specific stage
docker build --target builder -t myapp:builder .
docker run --rm -it myapp:builder sh
# Inspect filesystem state
```

**SSH into Failed CI (GitHub Actions):**
```yaml
- name: Setup tmate session
  if: failure()
  uses: mxschmitt/action-tmate@v3
  timeout-minutes: 30
```

**Local Reproduction:**
Ensure local Docker version matches CI:
```bash
# Check CI version
docker version

# Replicate exactly
docker build --no-cache --progress=plain -t local-test .
```

### Build Stability

**Network Flakiness:**
```dockerfile
# Retry failed downloads
RUN apt-get update || apt-get update || apt-get update
```

**Race Conditions in Parallel Builds:**
```yaml
# Disable BuildKit parallelism if experiencing race conditions
build:
  variables:
    BUILDKIT_PROGRESS: plain
    DOCKER_BUILDKIT: 1
  script:
    - docker build --progress=plain .
```

---

## Chapter Summary and Preview

In this chapter, we explored the integration of Docker containerization into Continuous Integration pipelines. We examined how `.dockerignore` files optimize build context transfer, reducing pipeline execution time by excluding unnecessary files. Multi-stage builds separate build-time dependencies from production runtime, creating smaller, more secure images essential for efficient registry storage and deployment. We analyzed layer caching strategies using registry backends and GitHub Actions cache to accelerate iterative builds, alongside the security and performance trade-offs of Docker-in-Docker versus rootless alternatives like Kaniko and Buildah. BuildKit integration provides advanced capabilities including secret mounts for credential protection, SSH mounts for private repositories, and parallel multi-platform builds. Image tagging strategies emphasizing immutable Git SHA references ensure traceability and prevent deployment ambiguity, while registry authentication patterns using short-lived tokens maintain security in automated environments. Finally, we addressed debugging techniques for ephemeral CI environments, including verbose logging, intermediate layer inspection, and local reproduction strategies.

**Key Takeaways:**
- Always implement comprehensive `.dockerignore` files to minimize build context; large contexts are the primary cause of slow CI builds
- Use multi-stage builds to separate compilation dependencies from runtime, targeting distroless or scratch images for production to minimize CVE exposure
- Prefer registry-based caching over local layer caching in cloud CI environments to share cache across runners and pipeline instances
- Avoid Docker-in-Docker privileged mode in security-sensitive environments; use Kaniko or Buildah for rootless, daemonless building
- Tag images with immutable identifiers (Git SHA) rather than floating tags (latest) to ensure reproducible deployments and enable reliable rollback procedures

**Next Chapter Preview:**
Chapter 22: Testing in CI transitions from building containers to validating their contents. We will explore unit testing within containerized environments, integration testing using service containers and Testcontainers, end-to-end testing strategies for complete application stacks, and parallel test execution to maintain rapid feedback loops. This chapter examines how to structure test suites that execute reliably in ephemeral CI environments, including test data management, database migration testing, and the generation of coverage reports that gate deployment quality. Understanding these testing patterns ensures that the Docker images built in this chapter meet functional and quality standards before progressing to deployment stages.